In [13]:
%load_ext autoreload
%autoreload 2

import json
from flask import Flask
from flask_cors import CORS, cross_origin
from pymongo import MongoClient
from bson.objectid import ObjectId
from db_interface import addMagresFile

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# MAGRES database
client = MongoClient(host='wigner.esc.rl.ac.uk')
ccpnc = client.ccpnc

In [12]:
ccpnc.collection_names()

[u'magresFilesFS',
 u'magresFiles',
 u'magresMetadata',
 u'magresData',
 u'magresIndex']

In [34]:
mfile = open('/home/phony_stark/Desktop/RAL/MagresViewHg/magresview/examples/ethanol-all.magres').read()
addMagresFile(mfile, chemname='thingNow', orcid={u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'})

5a6b61930c81be53f30a4470


1

In [17]:
def findById(fileID):
    return ccpnc.magresFiles.find({'_id': ObjectId(fileID)})

In [36]:
ff = ccpnc.magresData.find({"chemname": "Ethanol"})
# Get the first magres file in the query
mfiles = []
for f in ff:
    print f['chemname']
    print f['magresFilesID']
    mfile = findById(f['magresFilesID'])
    for m in mfile:
        mfiles.append(m['magres'])

Ethanol
5a0060c2320786662cf12bbe


In [38]:
ff = ccpnc.magresMetadata.find({"chemname": "thingNow"})

for f in ff:
    print f['version_history']

[{u'magresFilesID': ObjectId('5a6b61930c81be53f30a4470')}]


In [19]:
ff = ccpnc.magresData.find({"orcid.path": "0000-0003-4851-1144"})
for f in ff:
    print f['orcid']

{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}


In [6]:
from db_schema import magresDataSchema

In [12]:
d = f.copy()
del d['_id']
del d['magresFilesID']

d['csd-ref'] = {
    'refcode': 'C6H6OH',
    'digits': '01'
}

magresDataSchema.validate(d)

{u'chemname': u'Ethanol (additional file)',
 'csd-ref': {'digits': '01', 'refcode': 'C6H6OH'},
 u'doi': u'10.0001',
 u'orcid': {u'host': u'orcid.org',
  u'path': u'0000-0003-4851-1144',
  u'uri': u'http://orcid.org/0000-0003-4851-1144'},
 u'values': [{u'iso': [29.559937639131203,
    30.22618664854139,
    30.072254456072027,
    26.953990818833876,
    27.373946759128433,
    31.888119371185127],
   u'species': u'H'},
  {u'iso': [156.1229153499832, 109.35753044488325], u'species': u'C'},
  {u'iso': [267.0122765992025], u'species': u'O'}]}

In [8]:
import re

In [9]:
myre = re.compile('[0-9]{2}\Z')

In [10]:
myre.match('34')